## import modules

In [13]:
import pandas as pd
from scipy import stats
import numpy as np

## census data processing

In [14]:
# for population data
cols=['State', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F']
census=pd.read_excel('datasets/census.xlsx',engine='openpyxl',usecols=cols)
censusIndia=census.iloc[0,:]
census=census.loc[(census.Level=='STATE') & (census.TRU=='Total')]
census=census.append(censusIndia,ignore_index=True)
census.sort_values(by=['State'],axis=0,inplace=True)
census.reset_index(drop=True,inplace=True)

In [15]:
census.head(10)

,State,Level,Name,TRU,No_HH,TOT_P,TOT_M,TOT_F
0,0,India,India,Total,249501663,1210854977,623270258,587584719
1,1,STATE,JAMMU & KASHMIR,Total,2119718,12541302,6640662,5900640
2,2,STATE,HIMACHAL PRADESH,Total,1483280,6864602,3481873,3382729
3,3,STATE,PUNJAB,Total,5513071,27743338,14639465,13103873
4,4,STATE,CHANDIGARH,Total,241173,1055450,580663,474787
5,5,STATE,UTTARAKHAND,Total,2056975,10086292,5137773,4948519
6,6,STATE,HARYANA,Total,4857524,25351462,13494734,11856728
7,7,STATE,NCT OF DELHI,Total,3435999,16787941,8987326,7800615
8,8,STATE,RAJASTHAN,Total,12711146,68548437,35550997,32997440
9,9,STATE,UTTAR PRADESH,Total,33448035,199812341,104480510,95331831


## male-to-female

In [16]:
c18=pd.read_excel('datasets/C-18.xlsx',engine='openpyxl',skiprows=6,header=None)

In [18]:
STATE_NAMES=[]
for state in c18.iloc[:,2].values:
    if not (state in STATE_NAMES):
        STATE_NAMES.append(state)

In [20]:
useful_data=[]
for i,state in enumerate(STATE_NAMES):
    
    # here i is the state code
    male_pop=census[(census['State']==i) & (census['TRU']=='Total')]['TOT_M'].values[0]
    female_pop=census[(census['State']==i) & (census['TRU']=='Total')]['TOT_F'].values[0]
    
    # tri
    tri_male=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Total')].iloc[0,9]
    tri_female=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Total')].iloc[0,10]

    #bi
    bi_male=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Total')].iloc[0,6] - tri_male
    bi_female=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Total')].iloc[0,7] - tri_female

    #uni
    uni_male=male_pop-bi_male-tri_male
    uni_female=female_pop-bi_female-tri_female

    item={
        'state-code':i,
        'state-name':state,
        'male-to-female':male_pop/female_pop,
        'tri-male-to-female-ratio':tri_male/tri_female,
        'bi-male-to-female-ratio':bi_male/bi_female,
        'uni-male-to-female-ratio':uni_male/uni_female
    }

    useful_data.append(item)

In [22]:
df=pd.DataFrame(useful_data)

- main thing is how to do p-test for these values